In [1]:
import pandas as pd
import json
import re
from tqdm.auto import tqdm
import nltk
from nltk.corpus import wordnet
import matplotlib.pyplot as plt

/leonardo_scratch/large/userexternal/gpuccett/Repos/llm_newest/conda_venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df = pd.read_csv("/leonardo_scratch/large/userexternal/gpuccett/Repos/abstact_llm/data/Scores_Conc_and_Spec_for_study1.csv")

In [13]:
def compute_spec(x):
    hypernym_closure = lambda s: s.hypernyms()
    ss = wordnet.synsets(x, pos="n")
    if len(ss) == 0:
        return 0
    ss = ss[0]
    spec = len(list(ss.closure(hypernym_closure)))
    if spec == 0:
        ss = ss.instance_hypernyms()
        if not len(ss) == 0:
            ss = ss[0]
            spec = len(list(ss.closure(hypernym_closure)))
    spec = float(5 *  spec) / float(20)
    return spec

def normalize_score(x, _min, _max):
    return ((x - _min) / (_max - _min)) * (5.0 - 1.0) + 1.0

In [4]:
with open(
    # "/leonardo_scratch/large/userexternal/gpuccett/Repos/abstact_llm/generation_output_llama-3.1-8b-instruct-hf_You are a child of 8 years old learning in elementary school..jsonl"
    "/leonardo_scratch/large/userexternal/gpuccett/Repos/abstact_llm/generation_output_llama-3.1-405b-instruct-hf_You are a child of 3 years old learning about the world..jsonl"
) as jf:
    out = [json.loads(i) for i in jf.readlines()]

In [ ]:
def fix_record(_x):
    try:
        x = _x
        x = [j.strip() for j in re.split("\d\.|•|\*|-", x["generated_text"])]
        x = x if len(x[0].split()) == 1 else x[1:]
        y = x[-1].lower()
        seps = ["in this list", "this list"]
        for sep in seps:
            if sep in x[-1].lower():
                y = y.split(sep)[0]
                x[-1] = y.strip()
                break
        return x
    except:
        return None

new_out = [fix_record(i) for i in tqdm(out)]
new_out = [i for i in new_out if i is not None]
for i in out:
    i["list"] = fix_record(i)

for i in out:
    i["list_spec"] = [compute_spec(j) for j in i["list"] if j is not None]

: 

: 

In [6]:
id = 3
print("\t", out[id]["token"])
new_out[id]

	 flashback


['Things',
 'Memories',
 'Thoughts',
 'Recollections',
 'Flashback',
 'Movie flashback',
 'War flashback',
 'Traumatic flashback episode',
 'PTSD',
 'induced war flashback scene in a movie']

In [8]:
df.loc[:, "spec"] = df.token.apply(compute_spec)
_min, _max = df.spec.min(), df.spec.max()
df.loc[:, "norm_spec"] = df.spec.apply(lambda x: normalize_score(x, _min, _max))

/leonardo_scratch/large/userexternal/gpuccett/Repos/llm_newest/conda_venv/lib/python3.11/site-packages/nltk/corpus/reader/wordnet.py:604: UserWarning: Discarded redundant search for Synset('physical_entity.n.01') at depth 9
  for synset in acyclic_breadth_first(self, rel, depth):
/leonardo_scratch/large/userexternal/gpuccett/Repos/llm_newest/conda_venv/lib/python3.11/site-packages/nltk/corpus/reader/wordnet.py:604: UserWarning: Discarded redundant search for Synset('matter.n.03') at depth 6
  for synset in acyclic_breadth_first(self, rel, depth):
/leonardo_scratch/large/userexternal/gpuccett/Repos/llm_newest/conda_venv/lib/python3.11/site-packages/nltk/corpus/reader/wordnet.py:604: UserWarning: Discarded redundant search for Synset('physical_entity.n.01') at depth 7
  for synset in acyclic_breadth_first(self, rel, depth):
/leonardo_scratch/large/userexternal/gpuccett/Repos/llm_newest/conda_venv/lib/python3.11/site-packages/nltk/corpus/reader/wordnet.py:604: UserWarning: Discarded redun

In [9]:
df.iloc[:, 2:].corr()

,specificity1,specificity2,specificity3,spec,norm_spec
specificity1,1.000000,0.988183,0.847100,0.748963,0.748963
specificity2,0.988183,1.000000,0.822268,0.726260,0.726260
specificity3,0.847100,0.822268,1.000000,0.889530,0.889530
spec,0.748963,0.726260,0.889530,1.000000,1.000000
norm_spec,0.748963,0.726260,0.889530,1.000000,1.000000
